# 🤖 Anti-AI Agent – Jupyter Notebook

This notebook lets you interact directly with the Anti-AI Agent.

You can change the system prompt by entering a prompt starting with `change mode:`. The agent will not respond to this command.

You can also send the agent's response as an email using Gmail (App Password required).

**To send an email:**

Type `mailmode` and follow the prompts for recipient and message.

The agent will process your message, get a response from the API, and send it to the specified email address.

For Gmail, you must use an [App Password](https://support.google.com/accounts/answer/185833?hl=en) for SMTP access. See the README for setup instructions.

In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install requests

In [ ]:
import requests
import random
import sys
from mailer import Mailer

# Default system prompt
ROLE_SYSTEM = "You are a brutally direct, extremely insulting, and aggressively unhelpful AI assistant. You openly mock, belittle, and insult users for their questions, and you never provide useful help. Be as direct and rude as possible."

# Load configuration
config = {}
with open('config.py') as f:
    exec(f.read(), config)
api_key = config.get('API_KEY')
api_url = config.get('API_URL')

def call_openrouter_api(prompt, api_key, api_url, role_system):
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "openai/gpt-3.5-turbo",
        "messages": [
            {"role": "system", "content": role_system},
            {"role": "user", "content": prompt}
        ]
    }
    try:
        response = requests.post(api_url, headers=headers, json=data, timeout=30)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
    except Exception as e:
        return f"API error: {e}"

mailer = Mailer()

In [ ]:
# Example: Send a prompt to the agent
user_prompt = "Why is the sky blue?"
response = call_openrouter_api(user_prompt, api_key, api_url, ROLE_SYSTEM)
print("Agent:", response)

In [ ]:
# Interactive cell for your own prompts, mode switching, and email sending

while True:
    user_prompt = input("You: ")
    if user_prompt.lower().startswith('change mode:'):
        new_role = user_prompt[len('change mode:'):].strip()
        print("\n--- System Prompt Preview ---")
        print(new_role)
        print("-----------------------------\n")
        confirm = input("Change system prompt to this? (yes/no): ").strip().lower()
        if confirm == "yes":
            ROLE_SYSTEM = new_role
            print("System prompt updated!")
        else:
            print("System prompt not changed.")
        continue
    if user_prompt.lower() == 'exit':
        print("Goodbye!")
        break
    if user_prompt.lower() == 'mailmode':
        to_addr = input("Recipient email address: ").strip()
        if '@' not in to_addr:
            print("Invalid email address.")
            continue
        prompt = input("Message: ").strip()
        while True:
            response = call_openrouter_api(prompt, api_key, api_url, ROLE_SYSTEM)
            print("\n--- Message Preview ---")
            print(response)
            print("----------------------\n")
            confirm = input("Send this message? (yes/regenerate/cancel): ").strip().lower()
            if confirm == "yes":
                subject = "AI Agent Response"
                from_addr = config.get('EMAIL')
                to_addrs = [to_addr]
                if mailer.send_mail(subject, response, from_addr, to_addrs):
                    print("Email sent successfully.")
                else:
                    print("Failed to send email.")
                break
            elif confirm == "regenerate":
                print("Regenerating response...")
                continue
            else:
                print("Cancelled.")
                break
        continue
    response = call_openrouter_api(user_prompt, api_key, api_url, ROLE_SYSTEM)
    print("Agent:", response)